# Getting Started with AgentRun CLI

## Overview

In this tutorial, you will learn how to use **AgentRun CLI** to develop, package, build, and deploy AI agents to AgentCube. AgentRun CLI is a comprehensive developer tool that streamlines the complete agent lifecycle from local development to cloud deployment.

We will walk through a practical example using a **LangChain Agent** that provides tool-calling capabilities with Calculator and Weather tools via FastAPI.

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Practical/Hands-on                                                           |
| Agent type          | LangChain-based API Service                                                  |
| Framework           | LangChain + FastAPI                                                          |
| Language            | Python 3.8+                                                                  |
| Tutorial components | Local development, packaging, building, publishing, and invoking agents      |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Beginner-friendly                                                            |
| Tools used          | AgentRun CLI, Docker, LangChain                                              |

### What You'll Learn

* How to structure an AI agent project
* How to use AgentRun CLI commands (`pack`, `build`, `publish`, `invoke`, `status`)
* How to configure agent metadata
* How to build and deploy agents to AgentCube
* Best practices for agent development

### Tutorial Architecture

The complete workflow consists of five main stages:

1. **Development**: Create your agent code locally
2. **Packaging**: Use `agentrun pack` to prepare the workspace
3. **Building**: Use `agentrun build` to create container images
4. **Publishing**: Use `agentrun publish` to deploy to AgentCube
5. **Invocation**: Use `agentrun invoke` to test your deployed agent

```
┌─────────────┐     ┌──────────┐     ┌─────────┐     ┌─────────┐     ┌────────┐
│ Development │ --> │   Pack   │ --> │  Build  │ --> │ Publish │ --> │ Invoke │
│    Agent    │     │          │     │  Image  │     │   to    │     │ Agent  │
│    Code     │     │          │     │         │     │ AgentCube│    │        │
└─────────────┘     └──────────┘     └─────────┘     └─────────┘     └────────┘
```

## Prerequisites

Before starting this tutorial, ensure you have:

* **Python 3.8+** installed
* **Docker** installed and running (for local builds)
* **AgentRun CLI** installed
* Basic understanding of Python and HTTP APIs

## Step 1: Install AgentRun CLI

First, let's install the AgentRun CLI tool. You can install it from source or from PyPI (when available).

In [ ]:
# Install AgentRun CLI from the cli-agentrun directory
# Note: Adjust the path based on your setup
!pip install -e ../../ --quiet

Verify the installation:

In [ ]:
!agentrun --help

**Expected Output:**

You should see the AgentRun CLI help message showing all available commands:
- `pack` - Package agent into standardized workspace
- `build` - Build container image
- `publish` - Publish agent to AgentCube
- `invoke` - Invoke published agent
- `status` - Check agent status

## Step 2: Explore the Example Agent

Let's examine the LangChain agent we'll be deploying. This agent provides a practical example of an AI agent that uses LangChain's framework with tool-calling capabilities.

In [ ]:
# View the agent directory structure
!ls -la agent/

**Expected Output:**

```
main.py          # The main agent code
requirements.txt # Python dependencies
```

In [ ]:
# View a snippet of the agent code
!head -50 agent/main.py

### Understanding the Agent

The LangChain Agent provides four main endpoints:

1. **`GET /`** - Service information and health check
2. **`GET /health`** - Health check endpoint for container orchestration
3. **`GET /tools`** - List available tools (Calculator, Weather)
4. **`POST /invoke`** - Invoke the agent with a prompt

The agent uses LangChain framework and includes:
- **Calculator Tool**: Performs basic mathematical calculations (e.g., "2 + 2", "10 * 5")
- **Weather Tool**: Provides mock weather information for cities (Shanghai, Beijing, New York, London)
- **Mock Mode**: Works without OpenAI API key for testing purposes
- **Conversational Memory**: Maintains context across interactions
- **FastAPI**: Modern async web framework for HTTP endpoints

## Step 3: Test the Agent Locally (Optional)

Before deploying, you can test the agent locally to ensure it works correctly.

In [ ]:
# Start the agent in the background (this cell will start a background process)
import subprocess
import time
import signal

# Start the agent
proc = subprocess.Popen(
    ['python', 'agent/main.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(2)

print("Agent started on port 8080")
print("PID:", proc.pid)

In [ ]:
# Test the agent with sample requests
import requests
import json

try:
    # Test health endpoint
    response = requests.get('http://localhost:8080/health')
    print("Health Check:")
    print(json.dumps(response.json(), indent=2))
    print()
    
    # Test tools listing
    response = requests.get('http://localhost:8080/tools')
    print("Available Tools:")
    print(json.dumps(response.json(), indent=2))
    print()
    
    # Test agent invocation with calculator
    response = requests.post(
        'http://localhost:8080/invoke',
        json={'prompt': 'What is 25 multiplied by 4?'}
    )
    print("Agent Response (Calculator Query):")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the agent is running on port 8080")

**Expected Output:**

```json
Health Check:
{
  "status": "healthy"
}

Available Tools:
{
  "tools": [
    {
      "name": "Calculator",
      "description": "Useful for performing mathematical calculations. Input should be a mathematical expression like '2 + 2' or '10 * 5'."
    },
    {
      "name": "Weather",
      "description": "Useful for getting weather information for a city. Input should be a city name like 'Shanghai' or 'New York'."
    }
  ]
}

Agent Response (Calculator Query):
{
  "response": "[Mock Response] Received prompt: 'What is 25 multiplied by 4?'. This would normally use the Calculator tool to perform the calculation. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

Note: Without an OpenAI API key, the agent runs in mock mode. With a valid API key, it will use LangChain to process requests and call the appropriate tools.

In [ ]:
# Stop the local agent
try:
    proc.terminate()
    proc.wait(timeout=5)
    print("Agent stopped")
except:
    proc.kill()
    print("Agent killed")

## Step 4: Package the Agent with `agentrun pack`

Now let's package our agent using the AgentRun CLI. The `pack` command:

- Validates the agent structure
- Creates `agent_metadata.yaml` configuration file
- Generates a `Dockerfile` automatically
- Prepares the workspace for building

In [ ]:
# Package the agent
!agentrun pack \
    -f agent \
    --agent-name "langchain-agent" \
    --description "A LangChain agent with Calculator and Weather tools" \
    --language "python" \
    --entrypoint "python main.py" \
    --port 8080 \
    --build-mode "local" \
    --verbose

**Expected Output:**

```
✅ Successfully packaged agent: langchain-agent
📁 Workspace: /path/to/agent
📄 Metadata: /path/to/agent/agent_metadata.yaml
```

### Understanding the Generated Files

After packaging, let's examine the generated files:

In [ ]:
# View the generated agent_metadata.yaml
!cat agent/agent_metadata.yaml

**Expected Output:**

```yaml
agent_name: langchain-agent
description: A LangChain agent with Calculator and Weather tools
language: python
entrypoint: python main.py
port: 8080
build_mode: local
requirements_file: requirements.txt
```

This metadata file is the central configuration for your agent. It defines:
- **agent_name**: Unique identifier for your agent
- **description**: Human-readable description
- **language**: Programming language (python, java)
- **entrypoint**: Command to start the agent
- **port**: Port the agent listens on
- **build_mode**: Build strategy (local or cloud)
- **requirements_file**: Dependencies file

In [ ]:
# View the generated Dockerfile
!cat agent/Dockerfile

**Expected Output:**

```dockerfile
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

EXPOSE 8080

CMD ["python", "main.py"]
```

The Dockerfile is automatically generated based on your agent's language and configuration.

## Step 5: Build the Agent Image with `agentrun build`

Now let's build a container image for our agent. The `build` command:

- Builds a Docker image from your agent workspace
- Supports local Docker builds and cloud builds
- Updates metadata with build information
- Provides detailed build progress

In [ ]:
# Build the agent image
!agentrun build -f agent --verbose

**Expected Output:**

```
✅ Successfully built agent image: langchain-agent
🏷️  Tag: latest
📏 Size: ~200MB
```

The build process:
1. Reads the Dockerfile and agent_metadata.yaml
2. Builds the Docker image locally
3. Tags the image with the agent name
4. Updates metadata with build information

In [ ]:
# Verify the Docker image was created
!docker images | grep langchain-agent

**Expected Output:**

```
langchain-agent    latest    abc123def456    1 minute ago    200MB
```

## Step 6: Test the Built Container (Optional)

Before publishing to AgentCube, you can test the container locally to ensure it works correctly.

In [ ]:
# Run the container locally
!docker run -d -p 8080:8080 --name langchain-agent-test langchain-agent:latest

import time
time.sleep(3)  # Wait for container to start
print("Container started!")

In [ ]:
# Test the containerized agent
import requests
import json

try:
    response = requests.post(
        'http://localhost:8080/invoke',
        json={'prompt': 'What is the weather in Shanghai?'}
    )
    print("Containerized Agent Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Error: {e}")

**Expected Output:**

```json
{
  "response": "[Mock Response] Received prompt: 'What is the weather in Shanghai?'. This would normally use the Weather tool to fetch weather information. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

Note: The agent is running in mock mode. With a valid OpenAI API key, it would use the Weather tool to provide weather information.

In [ ]:
# Stop and remove the test container
!docker stop langchain-agent-test
!docker rm langchain-agent-test

## Step 7: Publish the Agent with `agentrun publish`

Now let's publish our agent to AgentCube. The `publish` command:

- Registers the agent with AgentCube
- Pushes the Docker image to a registry (if using cloud mode)
- Creates a deployment endpoint
- Updates metadata with deployment information

**Note**: In the MVP version, the publish command simulates AgentCube integration for demonstration purposes.

In [ ]:
# Publish the agent to AgentCube
!agentrun publish \
    -f agent \
    --version "v1.0.0" \
    --image-url "docker.io/myorg/langchain-agent" \
    --description "LangChain agent with Calculator and Weather tools" \
    --verbose

**Expected Output:**

```
✅ Successfully published agent: langchain-agent
🆔 Agent ID: agent-abc123
🌐 Endpoint: https://api.agentcube.example.com/agents/agent-abc123/invoke
```

### Understanding the Publish Process

The publish command:
1. Validates that the agent has been built
2. Tags and pushes the image to the specified registry
3. Registers the agent with AgentCube API
4. Receives an agent ID and endpoint URL
5. Updates the metadata with deployment information

In [ ]:
# View the updated metadata after publishing
!cat agent/agent_metadata.yaml

**Expected Output:**

```yaml
agent_name: langchain-agent
description: LangChain agent with Calculator and Weather tools
language: python
entrypoint: python main.py
port: 8080
build_mode: local
requirements_file: requirements.txt
version: v1.0.0
agent_id: agent-abc123
agent_endpoint: https://api.agentcube.example.com/agents/agent-abc123/invoke
image:
  repository_url: docker.io/myorg/langchain-agent
  tag: latest
  build_mode: local
```

## Step 8: Check Agent Status with `agentrun status`

You can check the status of your published agent at any time using the `status` command.

In [ ]:
# Check agent status
!agentrun status -f agent --verbose

**Expected Output:**

```
╭───────────────────────────────────────────────────────────────╮
│                       Agent Status                             │
├────────────┬──────────────────────────────────────────────────┤
│ Property   │ Value                                             │
├────────────┼──────────────────────────────────────────────────┤
│ Agent Name │ langchain-agent                                   │
│ Agent ID   │ agent-abc123                                      │
│ Version    │ v1.0.0                                            │
│ Language   │ python                                            │
│ Build Mode │ local                                             │
│ Endpoint   │ https://api.agentcube.example.com/agents/...     │
╰────────────┴──────────────────────────────────────────────────╯
```

## Step 9: Invoke the Published Agent with `agentrun invoke`

Finally, let's test our published agent by invoking it with a sample payload.

In [ ]:
# Invoke the agent with a calculator query
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "What is 15 plus 27?"}' \
    --verbose

**Expected Output:**

```
✅ Successfully invoked agent
📤 Response: {
  "response": "[Mock Response] Received prompt: 'What is 15 plus 27?'. This would normally use the Calculator tool to perform the calculation. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

With a valid OpenAI API key, the response would be:
```
✅ Successfully invoked agent
📤 Response: {
  "response": "The result of 15 plus 27 is 42.",
  "success": true,
  "error": null
}
```

In [ ]:
# Invoke with a weather query
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "What is the weather like in Beijing?"}' \
    --verbose

**Expected Output:**

```
✅ Successfully invoked agent
📤 Response: {
  "response": "[Mock Response] Received prompt: 'What is the weather like in Beijing?'. This would normally use the Weather tool to fetch weather information. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

With a valid OpenAI API key, the agent would use the Weather tool and respond:
```
✅ Successfully invoked agent
📤 Response: {
  "response": "The weather in Beijing is Cloudy, 18°C, with 45% humidity.",
  "success": true,
  "error": null
}
```

In [ ]:
# Invoke with a general query
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "Tell me about the available tools."}' \
    --verbose

**Expected Output:**

```
✅ Successfully invoked agent
📤 Response: {
  "response": "[Mock Response] Received prompt: 'Tell me about the available tools.'. This would normally process your request using LangChain with OpenAI. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

With a valid OpenAI API key, the agent would provide information about its tools:
```
✅ Successfully invoked agent
📤 Response: {
  "response": "I have access to two tools: a Calculator for performing mathematical calculations, and a Weather tool for getting weather information for various cities.",
  "success": true,
  "error": null
}
```

## Step 10: Using Python SDK (Advanced)

AgentRun CLI also provides a Python SDK for programmatic access. This is useful for CI/CD pipelines and automation.

In [ ]:
# Example of using the Python SDK
from pathlib import Path
from agentrun.runtime.pack_runtime import PackRuntime
from agentrun.runtime.build_runtime import BuildRuntime
from agentrun.runtime.publish_runtime import PublishRuntime
from agentrun.runtime.invoke_runtime import InvokeRuntime

# Initialize runtimes
workspace_path = Path("agent").resolve()

# Pack the agent
pack_runtime = PackRuntime(verbose=True)
pack_result = pack_runtime.pack(
    workspace_path,
    agent_name="langchain-agent-sdk",
    description="LangChain agent packaged using Python SDK"
)
print(f"Packed: {pack_result['agent_name']}")

# Build the agent
build_runtime = BuildRuntime(verbose=True)
build_result = build_runtime.build(workspace_path)
print(f"Built: {build_result['image_name']}")

# Publish the agent
publish_runtime = PublishRuntime(verbose=True)
publish_result = publish_runtime.publish(
    workspace_path,
    version="v1.0.0",
    image_url="docker.io/myorg/langchain-agent-sdk"
)
print(f"Published: {publish_result['agent_id']}")

# Invoke the agent
invoke_runtime = InvokeRuntime(verbose=True)
invoke_result = invoke_runtime.invoke(
    workspace_path,
    {"prompt": "What is 10 times 5?"},
    {}
)
print(f"Invocation result: {invoke_result}")

## Summary and Best Practices

### What We Learned

In this tutorial, we covered the complete AgentRun CLI workflow:

1. ✅ **Installed** AgentRun CLI
2. ✅ **Explored** a practical sentiment analysis agent
3. ✅ **Tested** the agent locally
4. ✅ **Packaged** the agent with `agentrun pack`
5. ✅ **Built** a container image with `agentrun build`
6. ✅ **Published** the agent with `agentrun publish`
7. ✅ **Checked** status with `agentrun status`
8. ✅ **Invoked** the agent with `agentrun invoke`
9. ✅ **Used** the Python SDK for programmatic access

### Best Practices

#### 1. Project Structure
```
my-agent/
├── main.py              # Agent entrypoint
├── requirements.txt     # Dependencies
├── README.md           # Documentation
├── agent_metadata.yaml # Generated by agentrun pack
└── Dockerfile          # Generated by agentrun pack
```

#### 2. Agent Development
- **Test locally first** before packaging and deploying
- **Implement health checks** at `/health` endpoint
- **Support both `text` and `prompt`** fields for flexibility
- **Return structured JSON** responses
- **Handle errors gracefully** with proper HTTP status codes
- **Add logging** for debugging and monitoring

#### 3. Metadata Configuration
- **Use descriptive names** for your agents
- **Provide clear descriptions** to help users understand your agent
- **Specify correct entrypoint** and port
- **Choose appropriate build mode** (local for development, cloud for production)
- **Use semantic versioning** for version numbers (v1.0.0, v1.1.0, etc.)

#### 4. Building and Deployment
- **Test container locally** before publishing
- **Use proper image naming** conventions (organization/agent-name)
- **Tag images with versions** for easier rollback
- **Keep images small** by using slim base images
- **Document dependencies** clearly in requirements.txt

#### 5. CI/CD Integration
- **Use Python SDK** for automation
- **Implement automated testing** before deployment
- **Version control** your agent code and metadata
- **Use environment variables** for configuration
- **Set up monitoring** for production agents

### Command Reference

| Command | Purpose | Example |
|---------|---------|----------|
| `agentrun pack` | Package agent workspace | `agentrun pack -f ./agent --agent-name my-agent` |
| `agentrun build` | Build container image | `agentrun build -f ./agent --verbose` |
| `agentrun publish` | Publish to AgentCube | `agentrun publish -f ./agent --version v1.0.0` |
| `agentrun invoke` | Test deployed agent | `agentrun invoke -f ./agent --payload '{"text": "test"}'` |
| `agentrun status` | Check agent status | `agentrun status -f ./agent` |

### Next Steps

Now that you understand the basics, you can:

1. **Enhance the sentiment agent** with more sophisticated NLP capabilities
2. **Add authentication** using custom headers
3. **Implement streaming responses** for long-running tasks
4. **Create multi-agent systems** with agent-to-agent communication
5. **Integrate with external APIs** and services
6. **Build custom agents** for your specific use cases
7. **Explore cloud build mode** for serverless deployments
8. **Set up monitoring and logging** for production agents

### Additional Resources

- [AgentRun CLI Documentation](../../README.md)
- [Quick Start Guide](../../QUICKSTART.md)
- [Example Agents](../../examples/)
- [AgentCube Main Project](https://github.com/volcano-sh/agentcube)

### Getting Help

If you encounter issues or have questions:

- Run commands with `--verbose` flag for detailed logging
- Check `agentrun <command> --help` for command-specific help
- Review the [troubleshooting guide](../../QUICKSTART.md#troubleshooting)
- Open an issue on [GitHub](https://github.com/volcano-sh/agentcube/issues)

## Cleanup (Optional)

If you want to clean up the resources created in this tutorial:

In [ ]:
# Remove Docker images
!docker rmi langchain-agent:latest
!docker rmi langchain-agent-sdk:latest

In [ ]:
# Remove generated files (optional)
!rm -f agent/agent_metadata.yaml
!rm -f agent/Dockerfile

# Congratulations! 🎉

You've successfully completed the AgentRun CLI tutorial and learned how to develop, package, build, publish, and invoke AI agents using AgentRun CLI!

Happy agent building! 🚀